In [1]:
%%HTML
<style>
.container { width: 100% }
</style>

# Parsing with a Visitor

This notebooks shows how we can parse and evaluate assignments using the visitor pattern.

In [2]:
import sys
import antlr4 

Our grammar is stored in the file `Expr.g4`.  Let's have a look:

In [3]:
!cat Expr.g4

grammar Expr; 

prog: stat+ ;

stat: 'print' '(' expr ')' ';' # Print
    | ID '=' expr ';'          # Assign
    ;

expr: expr '+' prod            # Add
    | expr '-' prod            # Sub
    | prod                     # ExprProd
    ;

prod: prod '*' fact            # Mul
    | prod '/' fact            # Div
    | fact                     # ProdFact
    ;

fact: INT                      # int
    | ID                       # id
    | '(' expr ')'             # parens
    ;

ID  : [a-zA-Z][a-zA-Z0-9]* ; // match identifiers
INT : [0-9]|[1-9][0-9]* ;    // match integers
WS  : [ \t\n\r]+ -> skip ;   // toss out whitespace


Note that this grammar does not contain any embedded actions.  However, we have <em style="color:blue">marked</em>
the grammar rules in order to be able to refer to them later.  For example, the grammar rule
```
stat: expr ';' # Print
```
has been marked as `Print`.  Marking is done using the character `#` followed by the name of the mark.
In addition to the scanner and the parser we generate a visitor using the option `-visitor` below.

In [4]:
!java -jar /usr/local/lib/antlr-4.8-complete.jar -Dlanguage=Python3 -visitor Expr.g4

The files `ExprLexer.py` and `ExprParser.py` contain the generated scanner and parser, respectively. 
Additionally we also have the file `ExprVisitor` which contains a generic visitor for our grammar. 
We have to import these files.

In [5]:
from ExprLexer   import ExprLexer
from ExprParser  import ExprParser
from ExprVisitor import ExprVisitor

The visitor `ExprVisitor` implements a lot of methods which traverse the parse tree without performing any actions.
In order to evaluate expressions, we have to override some of the methods of this visitor.  To this end we define the class
`MyVisitor`.  This class maintains the dictionary `Values` that maps variable names to values.

In [6]:
class MyVisitor(ExprVisitor):
    def __init__(self):
        self.Values = {}

In [7]:
def visitAssign(self, ctx):
    name  = ctx.ID().getText()
    value = self.visit(ctx.expr())
    self.Values[name] = value
    return value

MyVisitor.visitAssign = visitAssign
del visitAssign

In [8]:
def visitPrint(self, ctx):
    value = self.visit(ctx.expr())
    print(value)
    return 0
    
MyVisitor.visitPrint = visitPrint
del visitPrint

In [9]:
def visitAdd(self, ctx):
    lhs = self.visit(ctx.expr())
    rhs = self.visit(ctx.prod())
    return lhs + rhs

MyVisitor.visitAdd = visitAdd
del visitAdd

In [10]:
def visitSub(self, ctx):
    lhs = self.visit(ctx.expr())
    rhs = self.visit(ctx.prod())
    return lhs - rhs

MyVisitor.visitSub = visitSub
del visitSub

In [11]:
def visitMul(self, ctx):
    lhs = self.visit(ctx.prod())
    rhs = self.visit(ctx.fact())
    return lhs * rhs

MyVisitor.visitMul = visitMul
del visitMul

In [12]:
def visitDiv(self, ctx):
    lhs = self.visit(ctx.prod())
    rhs = self.visit(ctx.fact())
    return lhs // rhs

MyVisitor.visitDiv = visitDiv
del visitDiv

In [13]:
def visitParens(self, ctx):
    return self.visit(ctx.expr())

MyVisitor.visitParens = visitParens
del visitParens

In [14]:
def visitInt(self, ctx):
        return int(ctx.INT().getText())
    
MyVisitor.visitInt = visitInt
del visitInt

In [15]:
def visitId(self, ctx):
    name = ctx.ID().getText()
    return self.Values.get(name, 0)

MyVisitor.visitId = visitId
del visitId

In [16]:
data = \
'''
a = 3;
b = 4;
x = a * a + b * b;
c = 5;
print(c * c);
print(x);
'''

Now we can parse a file of simple statements.  The function `parser_file` takes the string `data` as its argument and
executes the statements in this string by calling the method `visit` of the visitor we have defined.

In [17]:
def parse_data(data):
    input_stream = antlr4.InputStream(data)
    lexer        = ExprLexer(input_stream)
    token_stream = antlr4.CommonTokenStream(lexer)
    parser       = ExprParser(token_stream)
    tree         = parser.prog()
    visitor      = MyVisitor()
    visitor.visit(tree)

In [18]:
parse_data(data)

25
25
